In [1]:
import os, sys

import pymongo as pm
import numpy as np
import scipy.stats as stats
import pandas as pd
import json
import re
import requests
from io import BytesIO
from PIL import Image, ImageFilter

from skimage import io, img_as_float
import base64

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('talk')
sns.set_style('white')

from IPython.display import clear_output
import importlib
import time

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

### set up paths

In [76]:
# directory & file hierarchy
proj_dir = os.path.abspath('..')
analysis_dir = os.getcwd()
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
stimuli_dir = os.path.join(proj_dir,'stimuli')
datavol_dir = '/data/datasets/semantic_mapping' ## path to data on nightingale
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,results_dir,'sketches'))
gallery_dir = os.path.abspath(os.path.join(proj_dir,results_dir,'gallery'))
map_dir = os.path.abspath(os.path.join(proj_dir,results_dir,'maps'))
utils_dir = os.path.join(proj_dir,'utils')
mask_dir = os.path.join(utils_dir, 'object_masks')

## add helpers to python path
if os.path.join(proj_dir,'utils') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'utils'))
    
import utils as u
import socket

## import dictionaries that map between shapenet ids and graphical conventions naming scheme
importlib.reload(u)
G2S = u.GC2SHAPENET
S2G = u.SHAPENET2GC

def make_dir_if_not_exists(dir_name):   
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    return dir_name

## create directories that don't already exist        
result = [make_dir_if_not_exists(i) for i in [results_dir,plot_dir,csv_dir,sketch_dir,gallery_dir, map_dir, mask_dir]]

### import object metadata

In [3]:
from objects import *

## convert dict to list
object_list = [shapenet_30afd2ef2ed30238aa3d0a2f00b54836,shapenet_30dc9d9cfbc01e19950c1f85d919ebc2,shapenet_4c1777173111f2e380a88936375f2ef4,
shapenet_3466b6ecd040e252c215f685ba622927,shapenet_38f87e02e850d3bd1d5ccc40b510e4bd,shapenet_3cf6db91f872d26c222659d33fd79709,
shapenet_3d7ebe5de86294b3f6bcd046624c43c9,shapenet_56262eebe592b085d319c38340319ae4,shapenet_1d1641362ad5a34ac3bd24f986301745,shapenet_1da9942b2ab7082b2ba1fdc12ecb5c9e,shapenet_2448d9aeda5bb9b0f4b6538438a0b930,
shapenet_23b0da45f23e5fb4f4b6538438a0b930,shapenet_2b5953c986dd08f2f91663a74ccd2338,shapenet_2e291f35746e94fa62762c7262e78952,
shapenet_2eaab78d6e4c4f2d7b0c85d2effc7e09,shapenet_309674bdec2d24d7597976c675750537]

## convert list of dicts to pd dataframe
O = pd.DataFrame(object_list)

### create and save out object segmentation maps

In [84]:
this_url = O.url.values[0]

for i,d in O.iterrows():

    ## get image data from URL
    response = requests.get(d['url'])
    img = Image.open(BytesIO(response.content)).convert('RGB')

    ## find zero pixels and convert to object mask
    nonzeros = np.array(img)!=0
    _mask = nonzeros.astype(np.uint8).mean(2)*255
    mask = Image.fromarray(_mask).convert('L')

    ## save out mask to mask_dir
    fname = d['subordinate'] + '-mask.png'
    out_path = os.path.join(mask_dir, fname)
    mask.save(out_path) 
    print('{} | Saved out mask for {}!'.format(i+1, d['subordinate']))
    clear_output(wait=True)

print('Done!')

Done!
